In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# ----- Dane kalibracyjne (HU -> rho_e/rho_wody) -----
materials = [
    ("Air", -1000, 0.001),
    ("Lung (avg)", -750, 0.26),
    ("Fat", -100, 0.95),
    ("Water", 0, 1.00),
    ("Muscle", 40, 1.05),
    ("Trabecular bone", 300, 1.30),
    ("Cortical bone", 1000, 1.70),
]
labels, HU, rho = zip(*materials)
x_nodes = np.array(HU, dtype=float)
y_nodes = np.array(rho, dtype=float)

# ----- Newton: divided differences -----
def newton_coeffs(x, y):
    x = np.asarray(x, dtype=float)
    c = np.array(y, dtype=float).copy()
    n = len(x)
    for k in range(1, n):
        c[k:n] = (c[k:n] - c[k-1:n-1]) / (x[k:n] - x[0:n-k])
    return c


def newton_eval(z, x, c):
    z = np.asarray(z, dtype=float)
    p = np.full_like(z, c[-1], dtype=float)
    for k in range(len(c) - 2, -1, -1):
        p = p * (z - x[k]) + c[k]
    return p

In [9]:
# ----- Obliczenia -----
# Newton
c_newton = newton_coeffs(x_nodes, y_nodes)


# Library method: Vandermonde (polyfit) -> ten sam wielomian co Newton (inna baza)
deg = len(x_nodes) - 1
coef_poly = np.polyfit(x_nodes, y_nodes, deg)

# Siatka do wykresu
x_plot = np.linspace(x_nodes.min(), x_nodes.max(), 1000)
y_newton = newton_eval(x_plot, x_nodes, c_newton)
y_poly   = np.polyval(coef_poly, x_plot)

